# Marker Research Hack

## Company homepage domain search

In [1]:
import pandas as pd

try:
    blacklist = open("domain_blacklist.txt", "r+").read().splitlines()
except:
    blacklist = ['wikipedia.com']  # make sure this mock code runs even if file is missing

# TODO - PERFORM ACTUAL DOMAIN SEARCH
domains = ['https://www.lonza.com/', 'https://www.boehringer-ingelheim.com/', 'wikipedia.com']

df = pd.DataFrame({'domain': [domain for domain in domains if domain not in blacklist]})

df.to_csv('domains.csv')
df

,domain
0,https://www.lonza.com/
1,https://www.boehringer-ingelheim.com/


## Keyword extraction 

In [2]:
domains = pd.read_csv('domains.csv')

# TODO - PERFORM ACTUAL KEYWORD EXTRACTION
df = pd.DataFrame({'keywords': [['molecules', 'gene'], ['psyshiary', 'health']],
                   'company_name': ['lonza', 'Boehringer Ingelheim']},
                 index=domains["domain"])
df.index.name = 'domain'

df.to_csv('extracted_keywords.csv')
df

,keywords,company_name
domain,,
https://www.lonza.com/,"[molecules, gene]",lonza
https://www.boehringer-ingelheim.com/,"[psyshiary, health]",Boehringer Ingelheim


## Area of works classification

In [3]:
keywords = pd.read_csv('extracted_keywords.csv')

def classifier_predict(keywords):
    # TODO - return actual trained-classifier predictions
    return {'ad': 0.2, 'ds': 0.2, 'dp': 0.6}

predictions = keywords["keywords"].map(lambda keywords: classifier_predict(keywords))

df = pd.DataFrame({'predictions': predictions.to_list()},
                 index=domains["domain"])
df.index.name = 'domain'

df.to_csv('area_of_works_predictions.csv')

df

,predictions
domain,
https://www.lonza.com/,"{'ad': 0.2, 'ds': 0.2, 'dp': 0.6}"
https://www.boehringer-ingelheim.com/,"{'ad': 0.2, 'ds': 0.2, 'dp': 0.6}"


## Interactive user interface 

In [4]:
keywords = pd.read_csv('extracted_keywords.csv')
area_of_works_predictions = pd.read_csv('area_of_works_predictions.csv')


"""
TODO
show a UI with buttons with functionality for flagging domains (which writes to domains_blacklist.txt) and
flagging kywords (which writes to keywords_blacklist.txt)
"""
pd.merge(keywords, area_of_works_predictions)


,domain,keywords,company_name,predictions
0,https://www.lonza.com/,"['molecules', 'gene']",lonza,"{'ad': 0.2, 'ds': 0.2, 'dp': 0.6}"
1,https://www.boehringer-ingelheim.com/,"['psyshiary', 'health']",Boehringer Ingelheim,"{'ad': 0.2, 'ds': 0.2, 'dp': 0.6}"
